# LangChain 에이전트

에이전트는 언어 모델과 도구를 결합하여 작업에 대해 추론하고, 사용할 도구를 결정하며, 솔루션을 향해 반복적으로 작업할 수 있는 시스템을 만듭니다.

`create_agent`는 프로덕션 수준의 에이전트 구현을 제공합니다.

## 사전 준비

환경 변수를 설정합니다.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("LangChain-V1-Tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChain-V1-Tutorial


## 에이전트의 작동 방식

에이전트는 목표를 달성하기 위해 도구를 반복적으로 실행합니다. 에이전트는 중단 조건이 충족될 때까지 실행되며, 모델이 최종 출력을 생성하거나 반복 제한에 도달하면 중단됩니다.

```
입력 -> 모델 -> 도구 호출 -> 도구 실행 -> 모델 -> ... -> 최종 출력
```

### 모델 (Model)

모델은 에이전트의 추론 엔진입니다. 정적 또는 동적 방식으로 지정할 수 있습니다.

#### 정적 모델

정적 모델은 에이전트 생성 시 한 번 구성되며 실행 중에는 변경되지 않습니다.

In [3]:
from langchain.agents import create_agent

# 모델 식별자 문자열을 사용한 간단한 방법
agent = create_agent("openai:gpt-4.1-mini", tools=[])

In [4]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# 모델 인스턴스를 직접 초기화하여 더 세밀한 제어
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,  # 응답의 무작위성 제어
    max_tokens=1000,  # 최대 생성 토큰 수
    timeout=30,  # 요청 타임아웃(초)
)

agent = create_agent(model, tools=[])

#### 동적 모델

동적 모델은 런타임에 현재 상태와 컨텍스트를 기반으로 선택됩니다. 이를 통해 정교한 라우팅 로직과 비용 최적화가 가능합니다.

In [5]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

# 기본 모델과 고급 모델 정의
basic_model = ChatOpenAI(model="gpt-4.1-mini")
advanced_model = ChatOpenAI(model="gpt-4.1")


@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """대화 복잡도에 따라 모델 선택"""
    print(request)
    message_count = len(request.state["messages"])

    # 긴 대화에는 고급 모델 사용
    if message_count > 10:
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)


agent = create_agent(
    model=basic_model, tools=[], middleware=[dynamic_model_selection]  # 기본 모델
)

In [ ]:
from langchain_teddynote.messages import stream_graph
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig

stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해줘")]
    },
    config=RunnableConfig(),
)

ModelRequest(model=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11700b890>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11700db50>, root_client=<openai.OpenAI object at 0x11700a310>, root_async_client=<openai.AsyncOpenAI object at 0x11700ced0>, model_name='gpt-4.1-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True), system_prompt=None, messages=[HumanMessage(content='머신러닝의 동작 원리에 대해서 설명해줘', additional_kwargs={}, response_metadata={}, id='68a7d2b8-fdc5-497c-ad3e-657a1daef059')], tool_choice=None, tools=[], response_format=None, state={'messages': [HumanMessage(content='머신러닝의 동작 원리에 대해서 설명해줘', additional_kwargs={}, response_metadata={}, id='68a7d2b8-fdc5-497c-ad3e-657a1daef059')]}, runtime=Runtime(context=None, store=None, stream_writer=<function Pregel.stream.<locals>.stream_writer at 0x114dfee80>, previous=None), model_settings={})

🔄 Node: model 🔄
- - - - - - - - - -

### 도구 (Tools)

도구는 에이전트에게 행동을 취할 수 있는 능력을 부여합니다. 

에이전트는 다음을 지원합니다.
- 순차적으로 여러 도구 호출
- 적절한 경우 병렬 도구 호출
- 이전 결과를 기반으로 한 동적 도구 선택
- 도구 재시도 로직 및 오류 처리
- 도구 호출 간 상태 지속성

#### 도구 정의

In [16]:
from langchain.tools import tool
from langchain.agents import create_agent


@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"


@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    # 일부러 오류 발생
    raise Exception("날씨 정보를 가져오는 중 오류가 발생하였습니다.")


# 도구 리스트를 에이전트에 전달
agent = create_agent("openai:gpt-4.1-mini", tools=[search, get_weather])

#### 도구 오류 처리

도구 오류 처리를 커스터마이징하려면 `@wrap_tool_call` 데코레이터를 사용하여 미들웨어를 생성합니다.

In [17]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage


@wrap_tool_call
def handle_tool_errors(request, handler):
    """도구 실행 오류를 커스텀 메시지로 처리"""
    try:
        return handler(request)
    except Exception as e:
        # 모델에 커스텀 오류 메시지 반환
        return ToolMessage(
            content=f"[에러] 도구 호출시 오류가 발생했어요. 다시 시도해주세요. ({str(e)})",
            tool_call_id=request.tool_call["id"],
        )


agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search, get_weather],
    middleware=[handle_tool_errors],
)

In [18]:
stream_graph(agent, inputs={"messages": [HumanMessage(content="서울 날씨 조회해줘")]})


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[에러] 도구 호출시 오류가 발생했어요. 다시 시도해주세요. (날씨 정보를 가져오는 중 오류가 발생하였습니다.)
🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
죄송합니다. 현재 서울의 날씨 정보를 가져오는 데 문제가 발생했습니다. 잠시 후에 다시 시도해 주시겠어요?

## 프롬프트

### 시스템 프롬프트

시스템 프롬프트를 제공하여 에이전트가 작업에 접근하는 방식을 형성할 수 있습니다.

In [19]:
agent = create_agent(
    "openai:gpt-4.1-mini",
    tools=[search, get_weather],
    system_prompt="You are a helpful assistant. Be concise and accurate.",
)

#### 동적 시스템 프롬프트

런타임 컨텍스트나 에이전트 상태를 기반으로 시스템 프롬프트를 수정해야 하는 고급 사용 사례의 경우 미들웨어를 사용할 수 있습니다.

- 핵심: `request.runtime.context`

In [24]:
from typing import TypedDict
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


class Context(TypedDict):
    answer_type: str


@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """사용자 역할에 따라 시스템 프롬프트 생성"""
    answer_type = request.runtime.context.get("answer_type", "default")
    base_prompt = "You are a helpful assistant."

    if answer_type == "default":
        return f"{base_prompt} Answer in Korean. 간결하게 답변해줘."
    elif answer_type == "sns":
        return f"{base_prompt} Answer in Korean. SNS 형식으로 답변해줘."
    elif answer_type == "article":
        return f"{base_prompt} Answer in Korean. 뉴스 기사 형식으로 답변해줘."

    return base_prompt


agent = create_agent(
    model="openai:gpt-4.1-mini",
    tools=[search],
    middleware=[user_role_prompt],
    context_schema=Context,
)

In [28]:
# 컨텍스트에 따라 시스템 프롬프트가 동적으로 설정됩니다
stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해줘")]
    },
    context=Context(answer_type="sns"),
)


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[머신러닝 동작 원리]🤖✨

머신러닝은 '데이터'를 통해 스스로 학습하고 예측하는 기술이에요!

1️⃣ 데이터 수집: 다양한 데이터를 모아요.
2️⃣ 전처리: 데이터를 깔끔하게 정리해요.
3️⃣ 모델 학습: 알고리즘이 데이터를 분석하며 규칙을 찾아요.
4️⃣ 평가: 모델의 성능을 체크해요.
5️⃣ 예측: 새로운 데이터에 대해 결과를 예측해요.

쉽게 말해, 경험을 통해 더 똑똑해지는 컴퓨터라고 보면 돼요! #머신러닝 #인공지능 #데이터사이언스

## 에이전트 호출

에이전트의 상태에 업데이트를 전달하여 에이전트를 호출할 수 있습니다. 모든 에이전트는 상태에 메시지 시퀀스를 포함합니다.

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's the weather in San Francisco?"}]},
    context={"user_role": "expert"},
)

print(result)

## 고급 개념

### 구조화된 출력

특정 형식으로 에이전트의 출력을 반환하고 싶을 때가 있습니다. LangChain은 `response_format` 매개변수를 통해 구조화된 출력 전략을 제공합니다.

#### ToolStrategy

`ToolStrategy`는 인공적인 도구 호출을 사용하여 구조화된 출력을 생성합니다. 도구 호출을 지원하는 모든 모델에서 작동합니다.

In [31]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


# 응답 스키마 정의
class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str


agent = create_agent(
    model="openai:gpt-4.1-mini", tools=[], response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567",
            }
        ]
    }
)

print(result["structured_response"])
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

name='John Doe' email='john@example.com' phone='(555) 123-4567'


#### ProviderStrategy

`ProviderStrategy`는 모델 제공자의 네이티브 구조화된 출력 생성을 사용합니다. 더 안정적이지만 네이티브 구조화된 출력을 지원하는 제공자(예: OpenAI)에서만 작동합니다.

In [32]:
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model="openai:gpt-4.1", response_format=ProviderStrategy(ContactInfo)
)

In [33]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567",
            }
        ]
    }
)

In [34]:
result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

### 메모리

에이전트는 메시지 상태를 통해 대화 기록을 자동으로 유지합니다. 대화 중에 추가 정보를 기억하기 위해 커스텀 상태 스키마를 사용하도록 에이전트를 구성할 수도 있습니다.

상태에 저장된 정보는 에이전트의 단기 메모리로 생각할 수 있습니다.

#### 미들웨어를 통한 상태 정의

커스텀 상태가 특정 미들웨어 훅 및 해당 미들웨어에 연결된 도구에서 액세스해야 하는 경우 미들웨어를 사용하여 커스텀 상태를 정의합니다.

In [ ]:
from typing import Any
from langchain.agents import AgentState
from langchain.agents.middleware import AgentMiddleware


# 커스텀 상태 스키마 정의
class CustomState(AgentState):
    user_preferences: dict


class CustomMiddleware(AgentMiddleware):
    state_schema = CustomState
    tools = []

    def before_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        # 모델 호출 전 커스텀 로직
        pass


agent = create_agent("openai:gpt-4.1-mini", tools=[], middleware=[CustomMiddleware()])

# 에이전트는 이제 메시지 외에 추가 상태를 추적할 수 있습니다
result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "I prefer technical explanations"}],
        "user_preferences": {"style": "technical", "verbosity": "detailed"},
    }
)

#### `state_schema`를 통한 상태 정의

도구에서만 사용되는 커스텀 상태를 정의하는 단축 방법으로 `state_schema` 매개변수를 사용합니다.

In [ ]:
from langchain.agents import AgentState


class CustomState(AgentState):
    user_preferences: dict


agent = create_agent("openai:gpt-4.1-mini", tools=[], state_schema=CustomState)

result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "I prefer technical explanations"}],
        "user_preferences": {"style": "technical", "verbosity": "detailed"},
    }
)

### 스트리밍

에이전트가 여러 단계를 실행하는 경우 시간이 걸릴 수 있습니다. 중간 진행 상황을 표시하기 위해 메시지가 발생하는 대로 스트리밍할 수 있습니다.

In [ ]:
for chunk in agent.stream(
    {
        "messages": [
            {"role": "user", "content": "Search for AI news and summarize the findings"}
        ]
    },
    stream_mode="values",
):
    # 각 청크에는 해당 시점의 전체 상태가 포함됩니다
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif hasattr(latest_message, "tool_calls") and latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

### 미들웨어

미들웨어는 실행의 다양한 단계에서 에이전트 동작을 커스터마이징하기 위한 강력한 확장성을 제공합니다.

**미들웨어 사용 사례:**

- 모델 호출 전 상태 처리 (예: 메시지 트리밍, 컨텍스트 주입)
- 모델 응답 수정 또는 검증 (예: 가드레일, 콘텐츠 필터링)
- 커스텀 로직으로 도구 실행 오류 처리
- 상태 또는 컨텍스트에 따른 동적 모델 선택
- 커스텀 로깅, 모니터링 또는 분석 추가